In [33]:
#Selenium import
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException  # 예외 처리 추가
from selenium.webdriver.chrome.options import Options # user-agent 변경
#pandas import
import pandas as pd
#standard library
import time, os, random

# function

In [34]:
# 가장 작은 div 레이아웃 가져오기
def find_div(elements):
    html = []
    css = []
    for element in elements:
        try:
            # 현재 요소를 포함하는 가장 가까운 div 찾기
            closest_div = element.find_element(By.XPATH, './ancestor::div[1]')
            html_code = closest_div.get_attribute('outerHTML')
            
            # CSS 스타일 가져오기
            css_style = driver.execute_script("""
                const styles = window.getComputedStyle(arguments[0]);
                const styleObject = {};
                for (let i = 0; i < styles.length; i++) {
                    const propName = styles[i];
                    styleObject[propName] = styles.getPropertyValue(propName);
                }
                return styleObject;
            """, closest_div)
            
            # 스타일 출력
            style_output = "CSS Style:\n"
            for style_name, style_value in css_style.items():  # 딕셔너리의 각 항목
                style_output += f"{style_name}: {style_value}\n"
            
            # 결과 출력
            print("HTML Code:")
            print(html_code)
            print(style_output)
            
            #결과값 저장
            html.append(html_code)
            css.append(style_output)
            
        except NoSuchElementException:
            print("가장 가까운 div 요소를 찾을 수 없습니다.")
            
    return html, css

In [35]:
def search_keyword(keywords):
    # keyword가 포함된 block 불러와서 csv 파일에 저장하는 함수

    # 결과 파일 존재 여부 확인
    output_file = "result.csv"
    file_exists = os.path.isfile(output_file)
    
    # XPath OR 조건 생성
    xpath_conditions = " or ".join([f"contains(text(), '{keyword}')" for keyword in keywords])
    # 최종 XPath
    xpath = f"//*[{xpath_conditions}]"
    # Selenium을 활용해 요소 찾기
    elements = driver.find_elements(By.XPATH, xpath)

    # 각 요소가 어떤 키워드를 포함하는지 확인하기 위한 리스트
    matched_keywords = []
    
    #3. div 레이아웃 가져오기
    html_blocks, css_blocks = find_div(elements)
    for html_elements in html_blocks:
        element_keywords = [keyword for keyword in keywords if keyword in html_elements]
        matched_keywords.append(", ".join(element_keywords))
    df = pd.DataFrame({'HTML': html_blocks, 'CSS': css_blocks, 'Matched Keywords': matched_keywords})
    
    #4. 데이터 저장
    #처음 저장 시에만 헤더를 추가, 이후에는 헤더 제외
    if not file_exists:
        df.to_csv(output_file, index=False, mode='w', header=True, encoding='utf-8')  # 첫 번째 URL
        file_exists = True  # 첫 번째 이후로는 헤더를 제외
    else:
        df.to_csv(output_file, index=False, mode='a', header=False, encoding='utf-8')  # 이후 URL

# Scrapper

In [36]:
# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_argument("Mozilla/5.0")  # 원하는 User-Agent 문자열로 변경

In [37]:
data = pd.read_csv('test.csv')
driver = webdriver.Chrome() # chrome driver의 경로가 파일 경로와 같아야 함

In [38]:
# Keyword 설정
price_disrupt = ["실적", "조건", "신한카드", "삼성카드", "KB국민카드", "현대카드", "롯데카드", "우리카드", "하나카드", "BC카드", "NH농협카드"]
hidden_info = ["카드", "세금", "수수료", "청소료", "부가세", "서비스세", "관세", "팁", "관리비", "청소비", "청소료", "정리비", "운영비", "중개료", "중개비"]
emotional_lang = ["마지막", "포기", "정말로", "정말", "놓치지", "기회", "지금", "바로", "한정", "수량", "곧", "마감", "임박", "가장", "너무", "망설"]
disguised_ad = ["AD", "ad", "Ad"]
trap_question = ["않으"]
false_sale = ["1+1 할인", "1+1할인", "원쁠원 할인", "원플원 할인", "원 쁠 원 할인", "원 플 원 할인"]

keywords_dict = [price_disrupt, hidden_info, emotional_lang, disguised_ad, trap_question, false_sale]

In [39]:
for url in data['URL']:
    
    #1.url open
    print(url)
    driver.get(url)

    time.sleep(5)
    #time.sleep(random.randrange(10, 20)) # 10초 이상 기다려서 로딩되는데 걸리는 시간을 주기

    for keyword in keywords_dict:
        search_keyword(keyword) # element 불러와서 저장

https://www.coupang.com/np/search?component=&q=%EB%B9%84%EB%B9%94%EB%A9%B4&channel=user
HTML Code:
<div id="help-coupang-ranking-search-sort"><span>쿠팡 랭킹순은 판매 실적, 고객 선호도, 상품 경쟁력 및<br>검색 정확도 등을 종합적으로 고려한 것입니다.</span><ul><li><span>판매 실적 :</span>상품의 구매 건수</li><li><span>고객 선호도 :</span>상품 및 판매자에 대한 고객평가 등</li><li><span>상품 경쟁력 :</span>상품 가격, 배송 기간 등</li><li><span>검색 정확도 :</span>검색어와 상품 간 연관도 등</li></ul><p>이 밖에 쿠팡 랭킹순에는 고객의 편의 및 만족도</p><p>향상을 위한 추가적인 요소가 수시로 반영될 수 있습니다.</p></div>
CSS Style:
-webkit-border-horizontal-spacing: 0px
-webkit-border-image: none
-webkit-border-vertical-spacing: 0px
-webkit-box-align: stretch
-webkit-box-decoration-break: slice
-webkit-box-direction: normal
-webkit-box-flex: 0
-webkit-box-ordinal-group: 1
-webkit-box-orient: horizontal
-webkit-box-pack: start
-webkit-box-reflect: none
-webkit-font-smoothing: antialiased
-webkit-line-break: auto
-webkit-line-clamp: none
-webkit-locale: "ko-KR"
-webkit-mask-box-image: none
-webkit-mask-box-image-outset: 0
-webkit-mask-

In [40]:
driver.quit()